# `quickmt` Benchmark

In [ ]:
from time import time

import datasets
from fire import Fire
from quickmt import M2m100Translator, NllbTranslator, OpusmtTranslator, Translator
from sacrebleu import BLEU, CHRF, TER

In [ ]:
bleu = BLEU()
chrf = CHRF()
ter = TER()

In [ ]:
src_lang = "cs"
tgt_lang = "en"
src_lang_flores = "ces_Latn"
tgt_lang_flores = "eng_Latn"

quickmt_model_path = f"../../../quickmt-models/quickmt-{src_lang}-{tgt_lang}"
opusmt_model_path = f"../../../ct2-models/opus-mt-{src_lang}-{tgt_lang}-ct2"

if src_lang == "en" and tgt_lang == "bn":
    opusmt_model_string = "sarwarshafee8709809365/opus-mt-en-bn"
elif src_lang == "en" and tgt_lang == "pt":
    opusmt_model_string = f"Helsinki-NLP/opus-mt-roa-en"    
elif src_lang == "el" and tgt_lang == "en":
    opusmt_model_string = f"Helsinki-NLP/opus-mt-tc-big-el-en"   
elif src_lang == "en" and tgt_lang == "ja":
    opusmt_model_string = f"Helsinki-NLP/opus-mt-{src_lang}-jap"  
elif src_lang == "pt" and tgt_lang == "en":
    opusmt_model_string = f"Helsinki-NLP/opus-mt-en-roa"
elif src_lang == "en" and tgt_lang == "lv":
    opusmt_model_string = "Helsinki-NLP/opus-mt-tc-big-en-lv"
else:
    opusmt_model_string = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"

nllb600_model_path = "../../../ct2-models/nllb-200-distilled-600M-ct2"
nllb1b_model_path = "../../../ct2-models/nllb-200-distilled-1.3B-ct2"
m2m100_418m_model_path = "../../../ct2-models/m2m100_418-ct2/"
m2m100_1b_model_path = "../../../ct2-models/m2m100_1.2B-ct2/"

compute_device = "cuda"

In [ ]:
try:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{src_lang_flores}-{tgt_lang_flores}",
    )
except:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{tgt_lang_flores}-{src_lang_flores}",
    )

src = []
ref = []
for i in flores["devtest"]:
    src.append(i[f"sentence_{src_lang_flores}"])
    ref.append(i[f"sentence_{tgt_lang_flores}"])

In [ ]:
with open(f"flores.{src_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in src]))
with open(f"flores.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in ref]))

## Quickmt

In [ ]:
translator = Translator(model_path=quickmt_model_path, device=compute_device)

In [ ]:
src[1]

In [ ]:
translator(src[1], beam_size=5)

In [ ]:
translator(src[1], sampling_temperature=1.2, beam_size=1, sampling_topk=50, sampling_topp=0.9)

In [ ]:
t1 = time()
mt = translator(src)
t2 = time()
t2-t1

In [ ]:
with open(f"quickmt.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
del translator

In [ ]:
comet = ! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t quickmt.{tgt_lang} --batch_size 32 --only_system

In [ ]:
{"bleu": round(bleu.corpus_score(mt, [ref]).score, 2),
"chrf2": round(chrf.corpus_score(mt, [ref]).score, 2)}

In [ ]:
quickmt_metrics = {
    "bleu": round(bleu.corpus_score(mt, [ref]).score, 2),
    "chrf2": round(chrf.corpus_score(mt, [ref]).score, 2),
    "comet22": 100*float(comet[-1].split(" ")[-1]),
    "Time (s)": t2-t1
}

quickmt_metrics

## OpusMT

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model Helsinki-NLP/opus-mt-ro-en --output_dir ./ct2-models/opus-mt-ro-en-ct2
ct2-transformers-converter --model Helsinki-NLP/opus-mt-en-ro --output_dir ./ct2-models/opus-mt-en-ro-ct2
ct2-transformers-converter --model Helsinki-NLP/opus-mt-tc-big-en-lv --output_dir ./ct2-models/opus-mt-en-lv-big-ct2
```


In [ ]:
opusmt_metrics = None 

translator = OpusmtTranslator(
    model_path=opusmt_model_path,
    model_string=opusmt_model_string,
    device=compute_device,
)

In [ ]:
translator(src[0])

In [ ]:
t1 = time()
mt = translator(src)
t2 = time()
t2-t1

In [ ]:
opusmt_bleu = round(bleu.corpus_score(mt, [ref]).score, 2)
opusmt_bleu

In [ ]:
opusmt_chrf = round(chrf.corpus_score(mt, [ref]).score, 2)
opusmt_chrf

In [ ]:
del translator

In [ ]:
with open(f"opusmt.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
comet = ! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t opusmt.{tgt_lang} --batch_size 32 --only_system

In [ ]:
opusmt_metrics = {
    "bleu": opusmt_bleu,
    "chrf2": opusmt_chrf,
    "comet22": 100*float(comet[-1].split(" ")[-1]),
    "Time (s)": t2-t1
}

opusmt_metrics

## NLLB-600M

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/nllb-200-distilled-600M --output_dir ../ct2-models/nllb-200-distilled-600M-ct2
```

In [ ]:
translator = NllbTranslator(model_path=nllb600_model_path, device=compute_device)

In [ ]:
t1 = time()
mt = translator(src, src_lang=src_lang_flores, tgt_lang=tgt_lang_flores)
t2 = time()

In [ ]:
t2-t1

In [ ]:
del translator

In [ ]:
nllb_600m_bleu = round(bleu.corpus_score(mt, [ref]).score, 2)
nllb_600m_bleu

In [ ]:
nllb_600m_chrf = round(chrf.corpus_score(mt, [ref]).score, 2)
nllb_600m_chrf

In [ ]:
with open(f"nllb-600m.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
comet = ! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t nllb-600m.{tgt_lang} --batch_size 32 --only_system

In [ ]:
nllb6M_metrics = {
    "bleu": nllb_600m_bleu,
    "chrf2": nllb_600m_chrf,
    "comet22": 100*float(comet[-1].split(" ")[-1]),
    "Time (s)": t2-t1
}
print(nllb6M_metrics)

## NLLB-1.3B

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/nllb-200-distilled-1.3B --output_dir ../ct2-models/nllb-200-distilled-1.3B-ct2
```

In [ ]:
translator = NllbTranslator(model_path=nllb1b_model_path, device=compute_device)

In [ ]:
t1 = time()
mt = translator(src, src_lang=src_lang_flores, tgt_lang=tgt_lang_flores)
t2 = time()

In [ ]:
nllb_1b_bleu = round(bleu.corpus_score(mt, [ref]).score, 2)
nllb_1b_bleu

In [ ]:
nllb_1b_chrf = round(chrf.corpus_score(mt, [ref]).score, 2)
nllb_1b_chrf

In [ ]:
with open(f"nllb-1.3b.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
del translator

In [ ]:
comet = ! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t nllb-1.3b.{tgt_lang} --batch_size 16 --only_system

In [ ]:
nllb1B_metrics = {
    "bleu": nllb_1b_bleu,
    "chrf2": nllb_1b_chrf,
    "comet22": 100*float(comet[-1].split(" ")[-1]),
    "Time (s)": t2-t1
}
print(nllb1B_metrics)


## M2M100-418M

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/m2m100_418M --output_dir ../ct2-models/m2m100_418-ct2/
ct2-transformers-converter --model facebook/m2m100_1.2B --output_dir ../ct2-models/m2m100_1.2B-ct2/
```

In [ ]:
translator = M2m100Translator(model_path=m2m100_418m_model_path, device=compute_device)

In [ ]:
t1 = time()
mt = translator(src, src_lang=src_lang, tgt_lang=tgt_lang)
t2 = time()

In [ ]:
round(bleu.corpus_score(mt, [ref]).score, 2)

In [ ]:
round(chrf.corpus_score(mt, [ref]).score, 2)

In [ ]:
with open(f"m2m100-418m.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
comet = ! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t m2m100-418m.{tgt_lang} --batch_size 32 --only_system

In [ ]:
m2m4M_metrics = {
    "bleu": round(bleu.corpus_score(mt, [ref]).score, 2),
    "chrf2": round(chrf.corpus_score(mt, [ref]).score, 2),
    "comet22": 100*float(comet[-1].split(" ")[-1]),
    "Time (s)": t2-t1
}
print(m2m4M_metrics)


## M2M100-1.2B

Be sure to export the models to `ctranslate2` format first, e.g.

```bash
ct2-transformers-converter --model facebook/m2m100_1.2B --output_dir ../ct2-models/m2m100_1.2B-ct2/
```

In [ ]:
translator = M2m100Translator(model_path=m2m100_1b_model_path, device=compute_device)

In [ ]:
t1 = time()
mt = translator(src, src_lang=src_lang, tgt_lang=tgt_lang)
t2 = time()

In [ ]:
round(bleu.corpus_score(mt, [ref]).score, 2)

In [ ]:
round(chrf.corpus_score(mt, [ref]).score, 2)

In [ ]:
with open(f"m2m100-1.2B.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in mt]))

In [ ]:
comet = ! ~/miniforge3/envs/comet/bin/comet-score -s flores.{src_lang} -r flores.{tgt_lang} -t m2m100-1.2B.{tgt_lang} --batch_size 32 --only_system

In [ ]:
m2m1B_metrics = {
    "bleu": round(bleu.corpus_score(mt, [ref]).score, 2),
    "chrf2": round(chrf.corpus_score(mt, [ref]).score, 2),
    "comet22": 100*float(comet[-1].split(" ")[-1]),
    "Time (s)": t2-t1
}
print(m2m1B_metrics)


## Results

In [ ]:
import pandas as pd

if opusmt_metrics:
    df = pd.DataFrame({f"quickmt/quickmt-{src_lang}-{tgt_lang}":quickmt_metrics,
    opusmt_model_string: opusmt_metrics,
    "facebook/nllb-200-distilled-600M": nllb6M_metrics,
    "facebook/nllb-200-distilled-1.3B": nllb1B_metrics,
    "facebook/m2m100_418M": m2m4M_metrics,
    "facebook/m2m100_1.2B":m2m1B_metrics})
else:
    df = pd.DataFrame({
        f"quickmt-{src_lang}-{tgt_lang}":quickmt_metrics,
        "facebook/nllb-200-distilled-600M": nllb6M_metrics,
        "facebook/nllb-200-distilled-1.3B": nllb1B_metrics,
        "facebook/m2m100_418M": m2m4M_metrics,
        "facebook/m2m100_1.2B":m2m1B_metrics})

df = df.apply(lambda x: [f"{i:.2f}" for i in x])

print(f"\n## {src_lang} -> {tgt_lang} flores-devtest metrics")
print()

print(df.T.to_markdown())

## cs -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-cs-en            |  39.63 |   65.65 |     88.23 |       1.39 |
| Helsinki-NLP/opus-mt-cs-en       |  35.01 |   62.72 |     86.54 |       3.58 |
| facebook/nllb-200-distilled-600M |  38.35 |   64.25 |     87.48 |      21.62 |
| facebook/nllb-200-distilled-1.3B |  40.12 |   65.79 |     88.33 |      37.57 |
| facebook/m2m100_418M             |  31.59 |   60.1  |     84.53 |      18.31 |
| facebook/m2m100_1.2B             |  36.83 |   63.56 |     87.1  |      34.18 |


## lv -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-lv-en            |  35.3  |   62.91 |     86.98 |       1.39 |
| Helsinki-NLP/opus-mt-lv-en       |  30.86 |   59.56 |     85.23 |       3.61 |
| facebook/nllb-200-distilled-600M |  32.14 |   59.13 |     84.62 |      21.92 |
| facebook/nllb-200-distilled-1.3B |  36.63 |   62.59 |     86.86 |      37.95 |
| facebook/m2m100_418M             |  27.01 |   56.61 |     81.66 |      18.57 |
| facebook/m2m100_1.2B             |  33.61 |   61.31 |     86.15 |      35.87 |

## en -> pl flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt-en-pl                    |  21.76 |   52.11 |     87.15 |       1.42 |
| facebook/nllb-200-distilled-600M |  19.22 |   49.72 |     86.34 |      28.52 |
| facebook/nllb-200-distilled-1.3B |  21.22 |   51.81 |     88.98 |      49.89 |
| facebook/m2m100_418M             |  16.43 |   46.97 |     81.78 |      21.74 |
| facebook/m2m100_1.2B             |  19.57 |   50.28 |     86.41 |      42.55 |

## pl -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-pl-en            |  27.46 |   57.18 |     85.04 |       1.46 |
| Helsinki-NLP/opus-mt-pl-en       |  25.55 |   55.39 |     83.8  |       4.01 |
| facebook/nllb-200-distilled-600M |  29.28 |   57.11 |     84.65 |      21.61 |
| facebook/nllb-200-distilled-1.3B |  30.99 |   58.77 |     86.04 |      37.64 |
| facebook/m2m100_418M             |  22.12 |   52.51 |     80.41 |      17.99 |
| facebook/m2m100_1.2B             |  27.13 |   56.36 |     84.48 |      35.01 |

## en -> el flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-el            |  28.86 |   54.94 |     88.87 |       1.48 |
| Helsinki-NLP/opus-mt-en-el       |  25    |   52.16 |     86.9  |       4.63 |
| facebook/nllb-200-distilled-600M |  24.98 |   52.32 |     88.15 |      33.38 |
| facebook/nllb-200-distilled-1.3B |  27.36 |   54.43 |     89.35 |      58.82 |
| facebook/m2m100_418M             |  21.98 |   49.09 |     84.37 |      27.13 |
| facebook/m2m100_1.2B             |  26.59 |   53.09 |     88.18 |      52.98 |

## en -> th flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-th            |   9.55 |   54.77 |     84.52 |       1.27 |
| facebook/nllb-200-distilled-600M |   5.8  |   43.33 |     79.26 |      43.09 |
| facebook/nllb-200-distilled-1.3B |   6.29 |   46.91 |     83.25 |      58.57 |
| facebook/m2m100_418M             |   7.29 |   40.07 |     72.95 |      22.94 |
| facebook/m2m100_1.2B             |   8.48 |   45.02 |     78.65 |      42.88 |

## th -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-th-en            |  29.32 |   58.4  |     87.15 |       1.34 |
| Helsinki-NLP/opus-mt-th-en       |  19.76 |   48.86 |     81.59 |       3.84 |
| facebook/nllb-200-distilled-600M |  26.54 |   54.97 |     85.26 |      22.27 |
| facebook/nllb-200-distilled-1.3B |  29.38 |   57    |     86.59 |      39.43 |
| facebook/m2m100_418M             |  16.57 |   47.88 |     77.69 |      20.1  |
| facebook/m2m100_1.2B             |  21.71 |   52.63 |     82.51 |      37.8  |

## en -> hu flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-hu            |  28.73 |   59.42 |     88.02 |       1.39 |
| Helsinki-NLP/opus-mt-en-hu       |  24.87 |   56.61 |     85.36 |       4    |
| facebook/nllb-200-distilled-600M |  22.87 |   54.59 |     86.68 |      26.41 |
| facebook/nllb-200-distilled-1.3B |  25.68 |   57.27 |     88.99 |      46.36 |
| facebook/m2m100_418M             |  21.03 |   52.71 |     81.27 |      22.32 |
| facebook/m2m100_1.2B             |  26.53 |   57.83 |     87.09 |      43.64 |


## hu -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-hu-en            |  35.03 |   62.44 |     87.78 |       1.24 |
| Helsinki-NLP/opus-mt-hu-en       |  30.92 |   59.48 |     86.34 |       3.65 |
| facebook/nllb-200-distilled-600M |  33.29 |   60.48 |     86.82 |      21.19 |
| facebook/nllb-200-distilled-1.3B |  35.91 |   62.63 |     88.21 |      37.45 |
| facebook/m2m100_418M             |  27.22 |   56.36 |     83.56 |      18.05 |
| facebook/m2m100_1.2B             |  33.08 |   60.71 |     86.93 |      35.6  |


## en -> ur flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-ur            |  20.79 |   48.53 |     77.69 |       1.44 |
| Helsinki-NLP/opus-mt-en-ur       |   6.74 |   28.66 |     58.99 |       4.43 |
| facebook/nllb-200-distilled-600M |  21.02 |   48.71 |     81.12 |      27.58 |
| facebook/nllb-200-distilled-1.3B |  22.41 |   49.92 |     81.94 |      46.79 |
| facebook/m2m100_418M             |  13.89 |   40.14 |     71.48 |      24.21 |
| facebook/m2m100_1.2B             |  13.74 |   39.45 |     71.01 |      47.14 |


## ur -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-ur-en            |  31.48 |   58.35 |     84.25 |       1.28 |
| Helsinki-NLP/opus-mt-ur-en       |  10.37 |   35.21 |     62.54 |       4.79 |
| facebook/nllb-200-distilled-600M |  33.02 |   60.04 |     86.21 |      21.2  |
| facebook/nllb-200-distilled-1.3B |  36.68 |   62.83 |     87.55 |      36.97 |
| facebook/m2m100_418M             |  15.65 |   43.64 |     69.94 |      19.58 |
| facebook/m2m100_1.2B             |  23.78 |   51.67 |     79.44 |      35.35 |

## en -> tr flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt-en-tr                    |  32.72 |   63.77 |     89.42 |       1.24 |
| facebook/nllb-200-distilled-600M |  24.23 |   57.76 |     87.91 |      22.93 |
| facebook/nllb-200-distilled-1.3B |  28.36 |   61.19 |     89.74 |      40.06 |
| facebook/m2m100_418M             |  20.82 |   54.46 |     84.47 |      19.85 |
| facebook/m2m100_1.2B             |  22.44 |   56.43 |     86.85 |      37.71 |

## tr -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-tr-en            |  39.5  |   65.03 |     89    |       1.28 |
| Helsinki-NLP/opus-mt-tr-en       |  32.19 |   59.43 |     87.03 |       3.5  |
| facebook/nllb-200-distilled-600M |  37.15 |   62.76 |     88.28 |      20.91 |
| facebook/nllb-200-distilled-1.3B |  39.33 |   64.88 |     89.27 |      36.35 |
| facebook/m2m100_418M             |  28.53 |   57.08 |     84.59 |      17.34 |
| facebook/m2m100_1.2B             |  33.1  |   60.38 |     86.82 |      34.1  |


## en -> vi flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-vi            |  43.68 |   60.75 |     87.52 |       1.5  |
| Helsinki-NLP/opus-mt-en-vi       |  27.57 |   46.45 |     76.22 |       4.67 |
| facebook/nllb-200-distilled-600M |  39.06 |   57.27 |     86.95 |      24.12 |
| facebook/nllb-200-distilled-1.3B |  41.27 |   59.04 |     88.11 |      42.3  |
| facebook/m2m100_418M             |  33.95 |   52.82 |     82.53 |      20.32 |
| facebook/m2m100_1.2B             |  39.46 |   57.45 |     86.56 |      38.65 |

## vi -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-vi-en            |  37.6  |   62.86 |     87.21 |       1.27 |
| Helsinki-NLP/opus-mt-vi-en       |  25.93 |   53.32 |     81.77 |       3.45 |
| facebook/nllb-200-distilled-600M |  34.99 |   60.35 |     86.13 |      24.71 |
| facebook/nllb-200-distilled-1.3B |  37.67 |   62.64 |     87.12 |      36.8  |
| facebook/m2m100_418M             |  27.63 |   55.39 |     82.79 |      17.61 |
| facebook/m2m100_1.2B             |  32.79 |   59.01 |     85.58 |      33.98 |


## en -> fa flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt-en-fa                    |  26.22 |   54.08 |     85.79 |       1.18 |
| facebook/nllb-200-distilled-600M |  20.59 |   49.58 |     84.44 |      22.77 |
| facebook/nllb-200-distilled-1.3B |  21.85 |   50.88 |     86.46 |      39.55 |
| facebook/m2m100_418M             |  19.95 |   48.23 |     81.46 |      19.71 |
| facebook/m2m100_1.2B             |  16.97 |   45.22 |     78.93 |      38.79 |

## fa -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt-fa-en                    |  37.57 |   63.37 |     87.76 |       1.16 |
| facebook/nllb-200-distilled-600M |  34.79 |   60.86 |     86.49 |      21.17 |
| facebook/nllb-200-distilled-1.3B |  37.91 |   63.39 |     87.82 |      36.9  |
| facebook/m2m100_418M             |  27.2  |   55.82 |     82.9  |      18.24 |
| facebook/m2m100_1.2B             |  29.12 |   56.39 |     83.5  |      35.14 |

## en -> id flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-id            |  48.69 |   71.95 |     91.02 |       1.13 |
| Helsinki-NLP/opus-mt-en-id       |  39.71 |   66.5  |     88.24 |       3.08 |
| facebook/nllb-200-distilled-600M |  43.74 |   69.06 |     90.47 |      19.16 |
| facebook/nllb-200-distilled-1.3B |  46.14 |   70.7  |     91.39 |      33.09 |
| facebook/m2m100_418M             |  36.77 |   63.8  |     87.13 |      16.93 |
| facebook/m2m100_1.2B             |  42.62 |   68.04 |     89.76 |      32.4  |

## id -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-id-en            |  44.5  |   68.78 |     89.35 |       1.19 |
| Helsinki-NLP/opus-mt-id-en       |  34.62 |   62.07 |     86.31 |       3.35 |
| facebook/nllb-200-distilled-600M |  42.26 |   66.89 |     88.67 |      21.13 |
| facebook/nllb-200-distilled-1.3B |  45.25 |   68.92 |     89.51 |      36.01 |
| facebook/m2m100_418M             |  33.14 |   60.91 |     84.85 |      17.37 |
| facebook/m2m100_1.2B             |  39.1  |   65.07 |     87.55 |      33.41 |

## en -> bn flores-devtest metrics

|                                      |   bleu |   chrf2 |   comet22 |   Time (s) |
|:-------------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-bn                |  19.27 |   53.34 |     84.68 |       1.17 |
| sarwarshafee8709809365/opus-mt-en-bn |   1.48 |   21.6  |     49.57 |      10.01 |
| facebook/nllb-200-distilled-600M     |  15.64 |   50.65 |     85.69 |      28.01 |
| facebook/nllb-200-distilled-1.3B     |  17.31 |   52.17 |     86.45 |      46.94 |
| facebook/m2m100_418M                 |   7.39 |   36.54 |     70.44 |      44.3  |
| facebook/m2m100_1.2B                 |  12    |   45.19 |     81.92 |      61.97 |

## bn -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-bn-en            |  32.91 |   59.69 |     86.99 |       1.37 |
| Helsink-NLP/opus-mt-bn-en        |  17.89 |   45.94 |     78.62 |       3.45 |
| facebook/nllb-200-distilled-600M |  33.51 |   59.73 |     87.48 |      21.01 |
| facebook/nllb-200-distilled-1.3B |  36.4  |   62.18 |     88.61 |      36.62 |
| facebook/m2m100_418M             |  23.84 |   52.67 |     82.94 |      20.86 |
| facebook/m2m100_1.2B             |  27.26 |   54.86 |     84.28 |      36.28 |

## pt -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-pt-en            |  48.67 |   71.47 |     89.09 |       1.24 |
| Helsink-NLP/opus-mt-roa-en       |  45.1  |   69.43 |     88.22 |       3.69 |
| facebook/nllb-200-distilled-600M |  48.77 |   71.3  |     89.2  |      21.42 |
| facebook/nllb-200-distilled-1.3B |  51.02 |   72.83 |     89.78 |      36.86 |
| facebook/m2m100_418M             |  40.01 |   66.14 |     86.01 |      17.8  |
| facebook/m2m100_1.2B             |  45.69 |   69.52 |     88.3  |      34.42 |


## en -> ru flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-ru            |  32.29 |   59.12 |     87.77 |       1.43 |
| Helsink-NLP/opus-mt-en-ru        |  26.59 |   54.91 |     85.26 |       4.37 |
| facebook/nllb-200-distilled-600M |  28.79 |   56.58 |     87.58 |      26.71 |
| facebook/nllb-200-distilled-1.3B |  31.5  |   58.63 |     89.26 |      46.57 |
| facebook/m2m100_418M             |  23.16 |   51.73 |     82.12 |      20.51 |
| facebook/m2m100_1.2B             |  28.88 |   56.61 |     87    |      41.15 |


## ru -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-ru-en            |  33.9  |   61.63 |     85.7  |       1.31 |
| Helsink-NLP/opus-mt-ru-en        |  30.04 |   58.23 |     83.97 |       3.72 |
| facebook/nllb-200-distilled-600M |  34.59 |   61.26 |     85.88 |      21.93 |
| facebook/nllb-200-distilled-1.3B |  36.99 |   63.04 |     86.59 |      38.12 |
| facebook/m2m100_418M             |  26.62 |   56.31 |     81.77 |      18.73 |
| facebook/m2m100_1.2B             |  32.01 |   60.3  |     85.01 |      35.99 |


## es -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-es-en            |  28.64 |   58.61 |     86.11 |       1.33 |
| Helsink-NLP/opus-mt-es-en        |  27.62 |   58.38 |     86.01 |       3.67 |
| facebook/nllb-200-distilled-600M |  30.02 |   59.71 |     86.55 |      21.99 |
| facebook/nllb-200-distilled-1.3B |  31.58 |   60.96 |     87.25 |      38.2  |
| facebook/m2m100_418M             |  22.85 |   55.04 |     82.9  |      18.83 |
| facebook/m2m100_1.2B             |  26.84 |   57.69 |     85.47 |      36.22 |


## en -> es flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-es            |  26.64 |   55.12 |     85.15 |       1.41 |
| Helsink-NLP/opus-mt-en-es        |  26.56 |   55.01 |     84.92 |       3.79 |
| facebook/nllb-200-distilled-600M |  27.2  |   55.68 |     85.82 |      24.12 |
| facebook/nllb-200-distilled-1.3B |  28.06 |   56.47 |     86.55 |      42.12 |
| facebook/m2m100_418M             |  22.48 |   51.72 |     81.05 |      19.65 |
| facebook/m2m100_1.2B             |  25.75 |   54.38 |     84.58 |      38.47 |

## en -> hi flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-hi            |  35.94 |   59.92 |     79.06 |       1.31 |
| Helsink-NLP/opus-mt-en-hi        |  13.53 |   35.51 |     60.13 |       3.81 |
| facebook/nllb-200-distilled-600M |  31.13 |   56.66 |     80.26 |      24.92 |
| facebook/nllb-200-distilled-1.3B |  32.43 |   57.64 |     81.06 |      43.26 |
| facebook/m2m100_418M             |  26.17 |   51.24 |     74.65 |      25.04 |
| facebook/m2m100_1.2B             |  26.57 |   51.88 |     76.09 |      46.5  |

## hi -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-hi-en            |  39.9  |   65.04 |     88.77 |       0.98 |
| Helsink-NLP/opus-mt-hi-en        |  18.83 |   45.9  |     75.81 |       4.38 |
| facebook/nllb-200-distilled-600M |  38.8  |   64.29 |     88.87 |      22.49 |
| facebook/nllb-200-distilled-1.3B |  41.71 |   66.67 |     89.69 |      38.91 |
| facebook/m2m100_418M             |  29.81 |   57.66 |     85    |      19.65 |
| facebook/m2m100_1.2B             |  33.79 |   60.21 |     86.3  |      38.33 |

## en -> it flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-it            |  30.52 |   59.72 |     87.55 |       1.06 |
| Helsink-NLP/opus-mt-en-it        |  27.58 |   57.66 |     85.1  |       4.42 |
| facebook/nllb-200-distilled-600M |  28.58 |   58.3  |     87.3  |      24.93 |
| facebook/nllb-200-distilled-1.3B |  30.92 |   59.89 |     88.52 |      43.8  |
| facebook/m2m100_418M             |  23.81 |   54.34 |     82.64 |      20.34 |
| facebook/m2m100_1.2B             |  27.22 |   57.19 |     86.23 |      39.98 |

## it -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-it-en            |  32.1  |   61.48 |     87.31 |       1.04 |
| Helsink-NLP/opus-mt-it-en        |  29.48 |   59.96 |     86.59 |       4.39 |
| facebook/nllb-200-distilled-600M |  33.49 |   61.97 |     87.39 |      22.16 |
| facebook/nllb-200-distilled-1.3B |  34.97 |   63.23 |     88.14 |      38.26 |
| facebook/m2m100_418M             |  25.92 |   56.94 |     83.14 |      18.61 |
| facebook/m2m100_1.2B             |  30.81 |   60.43 |     86.43 |      36.28 |

## en -> ja flores-devtest metrics

|                                  |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|--------:|----------:|-----------:|
| quickmt/quickmt-en-ja            |   42.04 |     89.08 |       1.38 |
| Helsink-NLP/opus-mt-en-ja        |    6.41 |     62.91 |       7.09 |
| facebook/nllb-200-distilled-600M |   30.00 |     86.64 |      26.05 |
| facebook/nllb-200-distilled-1.3B |   32.38 |     88.02 |      46.04 |
| facebook/m2m100_418M             |   32.73 |     85.09 |      23.29 |
| facebook/m2m100_1.2B             |   35.83 |     87.78 |      43.89 |

## ja -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-ja-en            |  27.91 |   57.06 |     87.29 |       1.00 |
| Helsink-NLP/opus-mt-ja-en        |  19.22 |   49.15 |     82.64 |       3.54 |
| facebook/nllb-200-distilled-600M |  24.05 |   53.39 |     85.84 |      22.5  |
| facebook/nllb-200-distilled-1.3B |  28.4  |   56.96 |     87.47 |      37.15 |
| facebook/m2m100_418M             |  18.82 |   49.55 |     82.59 |      18.27 |
| facebook/m2m100_1.2B             |  23.32 |   53.46 |     85.43 |      35.44 |

## en -> ar flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-ar            |  29.61 |   61.59 |     86.28 |       1    |
| Helsink-NLP/opus-mt-en-ar        |  23.02 |   54.74 |     83.21 |       4.37 |
| facebook/nllb-200-distilled-600M |  23.03 |   55.13 |     84.84 |      28.4  |
| facebook/nllb-200-distilled-1.3B |  26.48 |   58.1  |     86.23 |      48.8  |
| facebook/m2m100_418M             |  19.4  |   51.52 |     81.36 |      22.23 |
| facebook/m2m100_1.2B             |  13.56 |   43.43 |     75.32 |      45.77 |

## ar -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-ar-en            |  42.79 |   66.98 |     87.4  |       0.88 |
| Helsink-NLP/opus-mt-ar-en        |  34.22 |   61.26 |     84.5  |       3.78 |
| facebook/nllb-200-distilled-600M |  39.13 |   64.14 |     86.22 |      21.58 |
| facebook/nllb-200-distilled-1.3B |  42.29 |   66.55 |     87.55 |      37.7  |
| facebook/m2m100_418M             |  29.41 |   57.68 |     82.21 |      18.5  |
| facebook/m2m100_1.2B             |  29.77 |   56.7  |     80.77 |      36.23 |


## en -> ko flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-ko            |  14.97 |   36.96 |     87.1  |       1.36 |
| facebook/nllb-200-distilled-600M |  12.15 |   33.66 |     87.39 |      24.75 |
| facebook/nllb-200-distilled-1.3B |  13.23 |   35.62 |     88.39 |      40.29 |
| facebook/m2m100_418M             |   9.91 |   30.69 |     83.2  |      22.23 |
| facebook/m2m100_1.2B             |  11.35 |   33.26 |     85.65 |      41.15 |

## ko -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-ko-en            |  27.03 |   56.25 |     86.11 |       1.05 |
| Helsink-NLP/opus-mt-ko-en        |  20.78 |   50.39 |     83.06 |       3.62 |
| facebook/nllb-200-distilled-600M |  26.53 |   55.04 |     85.83 |      21.28 |
| facebook/nllb-200-distilled-1.3B |  29.61 |   57.58 |     87.24 |      37.42 |
| facebook/m2m100_418M             |  20.75 |   50.65 |     82.07 |      18.21 |
| facebook/m2m100_1.2B             |  24.59 |   54.17 |     85.15 |      34.82 |

## de -> en flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-de-en            |  44.21 |   68.83 |     88.89 |       1.03 |
| Helsink-NLP/opus-mt-de-en        |  40.04 |   66.16 |     87.68 |       3.47 |
| facebook/nllb-200-distilled-600M |  42.46 |   67.07 |     88.14 |      21.36 |
| facebook/nllb-200-distilled-1.3B |  44.44 |   68.75 |     89.08 |      37.58 |
| facebook/m2m100_418M             |  34.27 |   61.86 |     84.52 |      17.89 |
| facebook/m2m100_1.2B             |  40.34 |   65.99 |     87.67 |      35.27 |

## en -> de flores-devtest metrics

|                                  |   bleu |   chrf2 |   comet22 |   Time (s) |
|:---------------------------------|-------:|--------:|----------:|-----------:|
| quickmt/quickmt-en-de            |  40.17 |   66.26 |     86.91 |       1.1  |
| Helsink-NLP/opus-mt-en-de        |  36.06 |   63.53 |     84.63 |       3.72 |
| facebook/nllb-200-distilled-600M |  35.72 |   63.41 |     86.65 |      26.93 |
| facebook/nllb-200-distilled-1.3B |  38.61 |   65.01 |     87.99 |      46.57 |
| facebook/m2m100_418M             |  28.57 |   57.76 |     79.75 |      21.46 |
| facebook/m2m100_1.2B             |  36.24 |   63.37 |     85.82 |      41.5  |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-it-en            | 61.48 | 32.10   | 87.31   | 0.9      |
| Helsinki-NLP/opus-mt-it-en       | 59.96 | 29.48   | 86.59   | 4.36     |
| facebook/nllb-200-distilled-600M | 61.97 | 33.49   | 87.39   | 23.0     |
| facebook/nllb-200-distilled-1.3B | 63.23 | 34.97   | 88.14   | 40.4     |
| facebook/m2m100_418M             | 56.94 | 25.92   | 83.14   | 20.0     |
| facebook/m2m100_1.2B             | 60.43 | 30.81   | 86.43   | 38.1     |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-en-it            | 59.72 | 30.52   | 87.55   | 1.1      |
| Helsinki-NLP/opus-mt-en-it       | 57.66 | 27.58   | 85.10   | 5.28     |
| facebook/nllb-200-distilled-600M | 58.30 | 28.58   | 87.30   | 25.2     |
| facebook/nllb-200-distilled-1.3B | 59.89 | 30.92   | 88.52   | 46.0     |
| facebook/m2m100_418M             | 54.34 | 23.91   | 82.64   | 21.4     |
| facebook/m2m100_1.2B             | 57.19 | 27.22   | 86.23   | 41.2     |

| Model                            | chrf2 | bleu    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-hi-en            | 65.04 | 39.90   | 88.77   | 1.17     |
| Helsinki-NLP/opus-mt-hi-en       | 45.90 | 18.83   | 75.81   | 3.73     |
| facebook/nllb-200-distilled-600M | 64.90 | 38.80   | 88.87   | 22.4     |
| facebook/nllb-200-distilled-1.3B | 66.67 | 41.71   | 89.69   | 37.9     |
| facebook/m2m100_418M             | 57.66 | 29.81   | 85.00   | 18.7     |
| facebook/m2m100_1.2B             | 60.21 | 33.79   | 86.30   | 38.3     |

| Model                            | bleu | chrf2    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-en-hi            | 35.90 | 59.94   | 79.04  | 0.97      |
| Helsinki-NLP/opus-mt-en-hi       | 13.53 | 35.51   | 60.13  | 4.37      |
| facebook/nllb-200-distilled-600M | 31.13 | 56.66   | 80.26  | 26.1      |
| facebook/nllb-200-distilled-1.3B | 32.43 | 57.64   | 81.06  | 45.3      |
| facebook/m2m100_418M             | 26.17 | 51.24   | 74.65  | 26.2      |
| facebook/m2m100_1.2B             | 26.57 | 51.88   | 76.09  | 48.5      |

| Model                            | bleu | chrf2    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-es-en            |  |    |   |      |
| Helsinki-NLP/opus-mt-es-en       | 27.62 | 58.38   | 86.01  | 3.98     |
| facebook/nllb-200-distilled-600M | 30.02 | 59.71   | 86.55  | 22.9     |
| facebook/nllb-200-distilled-1.3B | 31.58 | 60.96   | 87.25  | 38.6     |
| facebook/m2m100_418M             | 22.85 | 55.04   | 82.90  | 19.0     |
| facebook/m2m100_1.2B             | 26.84 | 57.69   | 85.47  | 36.9     |

| Model                            | bleu | chrf2    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-en-es            |  |    |    |      |
| Helsinki-NLP/opus-mt-en-es       | 26.56 | 55.01   | 84.92   | 5.01     |
| facebook/nllb-200-distilled-600M | 27.20 | 55.68   | 85.82   | 26.2     |
| facebook/nllb-200-distilled-1.3B | 28.06 | 56.47   | 86.55   | 45.4     |
| facebook/m2m100_418M             | 22.48 | 51.72   | 81.05   | 20.6     |
| facebook/m2m100_1.2B             | 25.75 | 54.38   | 84.58   | 39.6     |

| Model                            | bleu | chrf2    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-ru-en            |  |    |   |      |
| Helsinki-NLP/opus-mt-ru-en       | 30.04 | 58.23    |    |       |
| facebook/nllb-200-distilled-600M | 34.60 | 61.26    |    |       |
| facebook/nllb-200-distilled-1.3B | 36.99 | 63.04    |    |       |
| facebook/m2m100_418M             | 26.62 | 56.31    |    |       |
| facebook/m2m100_1.2B             | 32.01 | 54.69    |    |       |

| Model                            | bleu | chrf2    | comet22 | Time (s) |
| -------------------------------- | ----- | ------- | ------- | -------- |
| quickmt/quickmt-en-ru            |  |    |    |      |
| Helsinki-NLP/opus-mt-en-ru       | 26.59  | 54.91   |     |      |
| facebook/nllb-200-distilled-600M | 28.79  | 56.58   |     |      |
| facebook/nllb-200-distilled-1.3B | 31.49  | 58.63   |     |      |
| facebook/m2m100_418M             | 23.16  | 51.73   |     |      |
| facebook/m2m100_1.2B             | 28.88  | 56.61   |     |      |



# en -> bn flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-en-bn                    |  19.26 |   53.36 |   84.68 |   0.95 |
| facebook/nllb-200-distilled-600M |  15.64 |   50.65 |   85.69 |  27.51 |
| facebook/nllb-200-distilled-1.3B |  17.31 |   52.17 |   86.45 |  45.61 |
| facebook/m2m100_418M             |   7.39 |   36.54 |   70.44 |  42.36 |
| facebook/m2m100_1.2B             |  12    |   45.19 |   81.92 |  61.64 |

## bn -> en flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-bn-en                    |  32.94 |   59.7  |   87    |   1.18 |
| Helsink-NLP/opusmt-bn-en         |  17.89 |   45.94 |   78.62 |   4.27 |
| facebook/nllb-200-distilled-600M |  33.51 |   59.73 |   87.48 |  20.97 |
| facebook/nllb-200-distilled-1.3B |  36.4  |   62.18 |   88.61 |  37.53 |
| facebook/m2m100_418M             |  23.84 |   52.67 |   82.94 |  22.55 |
| facebook/m2m100_1.2B             |  27.26 |   54.86 |   84.28 |  37.13 |

## en -> id flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-en-id                    |  48.71 |   71.96 |   91.03 |   0.97 |
| Helsink-NLP/opusmt-en-id         |  39.71 |   66.5  |   88.24 |   3.49 |
| facebook/nllb-200-distilled-600M |  43.74 |   69.06 |   90.47 |  19.23 |
| facebook/nllb-200-distilled-1.3B |  46.14 |   70.7  |   91.39 |  32.97 |
| facebook/m2m100_418M             |  36.77 |   63.8  |   87.13 |  17.66 |
| facebook/m2m100_1.2B             |  42.62 |   68.04 |   89.76 |  34.03 |

## id -> en flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-id-en                    |  44.58 |   68.82 |   89.36 |   0.94 |
| Helsink-NLP/opusmt-id-en         |  34.62 |   62.07 |   86.31 |   3.19 |
| facebook/nllb-200-distilled-600M |  42.26 |   66.89 |   88.67 |  20.65 |
| facebook/nllb-200-distilled-1.3B |  45.25 |   68.92 |   89.51 |  35.85 |
| facebook/m2m100_418M             |  33.14 |   60.91 |   84.85 |  17.28 |
| facebook/m2m100_1.2B             |  39.1  |   65.07 |   87.55 |  33.31 |

## pt -> en flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-pt-en                    |  48.64 |   71.48 |   89.09 |   0.96 |
| facebook/nllb-200-distilled-600M |  48.77 |   71.3  |   89.2  |  20.66 |
| facebook/nllb-200-distilled-1.3B |  51.02 |   72.83 |   89.78 |  35.85 |
| facebook/m2m100_418M             |  40.01 |   66.14 |   86.01 |  17.36 |
| facebook/m2m100_1.2B             |  45.69 |   69.52 |   88.3  |  33.53 |

## en -> pt flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-en-pt                    |  50.62 |   71.79 |   89.27 |   0.97 |
| facebook/nllb-200-distilled-600M |  47.68 |   70.28 |   89.05 |  23.75 |
| facebook/nllb-200-distilled-1.3B |  48.92 |   70.96 |   89.77 |  41.13 |
| facebook/m2m100_418M             |  41.14 |   65.85 |   85.49 |  19.08 |
| facebook/m2m100_1.2B             |  46.56 |   69.41 |   88.53 |  37.42 |



## en -> ja flores-devtest metrics
|                                  |   chrfs |   comet |   time |
|:---------------------------------|--------:|--------:|-------:|
| quickmt-en-ja                    |   42.04 |   89.08 |   1.21 |
| Helsink-NLP/opusmt-en-ja         |    6.41 |   62.91 |   6.97 |
| facebook/nllb-200-distilled-600M |   30    |   86.64 |  25.17 |
| facebook/nllb-200-distilled-1.3B |   32.38 |   88.02 |  44.36 |
| facebook/m2m100_418M             |   32.73 |   85.09 |  22.55 |
| facebook/m2m100_1.2B             |   35.83 |   87.78 |  42.18 |

## ja -> en flores-devtest metrics

|                                  |   bleu |   chrfs |   comet |   time |
|:---------------------------------|-------:|--------:|--------:|-------:|
| quickmt-ja-en                    |  27.91 |   57.06 |   87.29 |   0.86 |
| Helsink-NLP/opusmt-ja-en         |  19.22 |   49.15 |   82.64 |   3.52 |
| facebook/nllb-200-distilled-600M |  24.05 |   53.39 |   85.84 |  22.01 |
| facebook/nllb-200-distilled-1.3B |  28.4  |   56.96 |   87.47 |  36.6  |
| facebook/m2m100_418M             |  18.82 |   49.55 |   82.59 |  17.88 |
| facebook/m2m100_1.2B             |  23.32 |   53.46 |   85.43 |  34.81 |
